# Web Scraping for Indeed.com & Predicting Salaries

In this project, we will practice two major skills: collecting data by scraping a website and then building a binary predictor with Logistic Regression.

We are going to collect salary information on data science jobs in a variety of markets. Then using the location, title and summary of the job we will attempt to predict the salary of the job. For job posting sites, this would be extraordinarily useful. While most listings DO NOT come with salary information (as you will see in this exercise), being to able extrapolate or predict the expected salaries from other listings can help guide negotiations.

Normally, we could use regression for this task; however, we will convert this problem into classification and use Logistic Regression.

- Question: Why would we want this to be a classification problem?
- Answer: While more precision may be better, there is a fair amount of natural variance in job salaries - predicting a range be may be useful.

Therefore, the first part of the assignment will be focused on scraping Indeed.com. In the second, we'll focus on using listings with salary information to build a model and predict additional salaries.

### Scraping job listings from Indeed.com

We will be scraping job listings from Indeed.com using BeautifulSoup. Luckily, Indeed.com is a simple text page where we can easily find relevant entries.

First, look at the source of an Indeed.com page: (http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10")

Notice, each job listing is underneath a `div` tag with a class name of `result`. We can use BeautifulSoup to extract those. 

#### Setup a request (using `requests`) to the URL below. Use BeautifulSoup to parse the page and extract all results (HINT: Look for div tags with class name result)

In [4]:
URL = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10"

In [5]:
import requests
import bs4
from bs4 import BeautifulSoup
from IPython.display import HTML
import re
import numpy as np
import pandas as pd

In [6]:
## YOUR CODE HERE

i = requests.get(URL)
i_soup = BeautifulSoup(i.content, "lxml")
i_soup

<!DOCTYPE html>\n<html lang="en">\n<head>\n<meta content="text/html;charset=unicode-escape" http-equiv="content-type"/>\n<!-- pll --><script src="/s/4a89ac5/en_US.js" type="text/javascript"></script>\n<link href="/s/a6a334e/jobsearch_all.css" rel="stylesheet" type="text/css"/>\n<link href="http://rss.indeed.com/rss?q=data+scientist+%2420%2C000&amp;l=New+York" rel="alternate" title="Data Scientist $20,000 Jobs, Employment in New York State" type="application/rss+xml"/>\n<link href="/m/jobs?q=data+scientist+%2420%2C000&amp;l=New+York" media="handheld" rel="alternate"/>\n<script type="text/javascript">\n    \n    window['closureReadyCallbacks'] = [];\n\n    function call_when_jsall_loaded(cb) {\n        if (window['closureReady']) {\n            cb();\n        } else {\n            window['closureReadyCallbacks'].push(cb);\n        }\n    }\n</script>\n<script src="/s/0579f05/jobsearch-all-compiled.js" type="text/javascript"></script>\n<script type="text/javascript">\nvar pingUrlsForGA = 

Let's look at one result more closely. A single `result` looks like

```
<div class=" row result" data-jk="2480d203f7e97210" data-tn-component="organicJob" id="p_2480d203f7e97210" itemscope="" itemtype="http://schema.org/JobPosting">
<h2 class="jobtitle" id="jl_2480d203f7e97210">
<a class="turnstileLink" data-tn-element="jobTitle" onmousedown="return rclk(this,jobmap[0],1);" rel="nofollow" target="_blank" title="AVP/Quantitative Analyst">AVP/Quantitative Analyst</a>
</h2>
<span class="company" itemprop="hiringOrganization" itemtype="http://schema.org/Organization">
<span itemprop="name">
<a href="/cmp/Alliancebernstein?from=SERP&amp;campaignid=serp-linkcompanyname&amp;fromjk=2480d203f7e97210&amp;jcid=b374f2a780e04789" target="_blank">
    AllianceBernstein</a></span>
</span>
<tr>
<td class="snip">
<nobr>$117,500 - $127,500 a year</nobr>
<div>
<span class="summary" itemprop="description">
C onduct quantitative and statistical research as well as portfolio management for various investment portfolios. Collaborate with Quantitative Analysts and</span>
</div>
</div>
</td>
</tr>
</table>
</div>
```

While this has some more verbose elements removed, we can see that there is some structure to the above:
- The salary is available in a `nobr` element inside of a `td` element with `class='snip`.
- The title of a job is in a link with class set to `jobtitle` and a `data-tn-element="jobTitle`.  
- The location is set in a `span` with `class='location'`. 
- The company is set in a `span` with `class='company'`. 

### Write 4 functions to extract each item: location, company, job, and salary.

example: 
```python
def extract_location_from_result(result):
    return result.find ...
```


- Make sure these functions are robust and can handle cases where the data/field may not be available.
- Test the functions on the results above

In [7]:
## YOUR CODE HERE

def mark_sponsored(soup_file):
    sponsored_col = []
    for result in soup_file.findAll('div', {'class' : re.compile('row.*')}):
        if len(result.findAll('span', class_='sdn')) > 0 :
            entry = 'True'
            sponsored_col.append(entry)
        else:
            entry = 'False'
            sponsored_col.append(entry)
    return sponsored_col

def extract_location_from_result(soup_file):
    location_col = []
    for d in soup_file.findAll('span', class_='location'):
        entry = d.text
        location_col.append(entry)
    return location_col

def extract_company_from_result(soup_file):
    company_col = []
    for d in soup_file.findAll('span', class_='company'):
        entry = d.text
        company_col.append(entry)
    return company_col

def extract_job_from_result(soup_file):
    job_col = []
    for result in soup_file.findAll('a', {'data-tn-element' : re.compile('job.*')}):
        entry = result.text
        job_col.append(entry)
    return job_col

def extract_salary_from_result(soup_file):
    salary_col = []
    for result in soup_file.findAll('div', {'class' : re.compile('row.*')}):
        sal = result.findAll('nobr')
        if len(sal) > 0 :
            rstring = str(sal)
#               if '$' in rstring:
            salary_col.append(rstring)
#         else:
#                     del rstring
        else:
            salary_col.append(np.nan)
    return salary_col


# 
#         if len(result.findAll('span', class_='sdn')) > 0 :
#             entry = 'True'
#             sponsored_col.append(entry)
#         else:
#             entry = 'False'
#             sponsored_col.append(entry)
#     return sponsored_col

In [8]:
print mark_sponsored(i_soup)
print
print extract_job_from_result(i_soup)
print
print extract_company_from_result(i_soup)
print
print extract_location_from_result(i_soup)
print 
print extract_salary_from_result(i_soup)

['True', 'True', 'True', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'True', 'True']

[u'Data Scientist', u'Data Scientist & Analytics Developer', u'Sr. Data Scientist, Learning Services', u'Data Engineer, Analytics Technology', u'Senior Data Scientist', u'Director of Data Science and Analytics', u'Data Scientist', u'Data Scientist', u'Data Scientist', u'VP - Quantitative Analyst', u'Data Scientist', u'Data Scientist', u'Data Scientist', u'Data Scientist \u2013 Predictive Analytics', u'Junior Data Scientist']

[u'\n    AbilTo, Inc', u'\n    Lancer Insurance Company', u'\n\n    Chegg', u'\n\n\n    NBCUniversal\n', u'\n\n    Spreemo\n', u'\n\n    Paperless Post\n', u'\n\n    Dia&Co\n', u'\n\n    Tapad\n', u'\n\n    Oliver James Associates\n', u'\n\n\n    Barclays\n', u'\n\n    SecurityScorecard\n', u'\n\n\n    Crisis Text Line\n', u'\n\n    Tilting Point\n', u'\n\n    Gallup', u'\n\n    UncommonGoods']

[u'New York, NY', u'Long Beach, NY', u'

In [9]:
def build_jobpost_db(soup_file):

    company = extract_company_from_result(soup_file) 
    job = extract_job_from_result(soup_file)
    location = extract_location_from_result(soup_file)
    salary = extract_salary_from_result(soup_file)
    sponsored = mark_sponsored(soup_file)
    
    jobdf = pd.DataFrame({'Job' : job, 'Company' : company, 'Location' : location, 'Salary' : salary, 'Sponsored' : \
                                                                                          sponsored})
    
    return jobdf

In [10]:
build_jobpost_db(i_soup)

,Company,Job,Location,Salary,Sponsored
0,"\n AbilTo, Inc",Data Scientist,"New York, NY",NaN,True
1,\n Lancer Insurance Company,Data Scientist & Analytics Developer,"Long Beach, NY",NaN,True
2,\n\n Chegg,"Sr. Data Scientist, Learning Services","New York, NY",NaN,True
3,\n\n\n NBCUniversal\n,"Data Engineer, Analytics Technology","New York, NY",NaN,False
4,\n\n Spreemo\n,Senior Data Scientist,"New York, NY 10012 (Little Italy area)",NaN,False
5,\n\n Paperless Post\n,Director of Data Science and Analytics,"New York, NY",NaN,False
6,\n\n Dia&Co\n,Data Scientist,"New York, NY",NaN,False
7,\n\n Tapad\n,Data Scientist,"New York, NY",NaN,False
8,\n\n Oliver James Associates\n,Data Scientist,"New York, NY","[<nobr>$170,000 - $200,000 a year</nobr>]",False
9,\n\n\n Barclays\n,VP - Quantitative Analyst,"New York, NY",NaN,False


Now, to scale up our scraping, we need to accumulate more results. We can do this by examining the URL above.

- "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10"

There are two query parameters here we can alter to collect more results, the `l=New+York` and the `start=10`. The first controls the location of the results (so we can try a different city). The second controls where in the results to start and gives 10 results (thus, we can keep incrementing by 10 to go further in the list).

#### Complete the following code to collect results from multiple cities and starting points. 
- Enter your city below to add it to the search
- Remember to convert your salary to U.S. Dollars to match the other cities if the currency is different

In [16]:

max_results_per_city = 100

results = []

for city in set(['New+York', 'Chicago', 'San+Francisco', 'Austin', 'Denver', 'Las+Vegas']):
    for start in range(0, max_results_per_city, 10):
        URL = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l={city}&start={start}".format(**locals())
        # Grab the results from the request (as above)
        i = requests.get(URL)
        i_soup = BeautifulSoup(i.content, "lxml")
        # Append to the full set of results
        results.append(i_soup)


#### Use the functions you wrote above to parse out the 4 fields - location, title, company and salary. Create a dataframe from the results with those 4 columns.

In [73]:
## YOUR CODE HERE
list_of_dfs = []

for soup_file in results:
    list_of_dfs.append(build_jobpost_db(soup_file))

jobsdf = pd.concat(list_of_dfs)
jobsdf = jobsdf.reset_index()
    
jobsdf

,index,Company,Job,Location,Salary,Sponsored
0,0,\n Enterprise Consulting Services,Data Scientist,"Chicago, IL",NaN,True
1,1,\n\n HERE,Senior or Mid-level Data Scientist,"Chicago, IL",NaN,True
2,2,"\n\n Anthem, Inc.",Data Scientist or Data Scientist Lead(Healthca...,"Chicago, IL",NaN,True
3,3,"\n\n\n Anthem, Inc.\n",Data Scientist or Data Scientist Lead(Healthca...,"Chicago, IL 60601 (Loop area)",NaN,False
4,4,\n\n\n Northwestern University\n,Research Data Analyst,"Chicago, IL",NaN,False
5,5,\n\n\n Kaplan\n,Senior Data Scientist,"Chicago, IL",NaN,False
6,6,\n\n Azavar\n,Analyst-Data Analysis,"Chicago, IL",NaN,False
7,7,\n\n\n Workbridge Associates\n,Mid-Level Data Scientist,"Chicago, IL","[<nobr>$105,000 a year</nobr>]",False
8,8,\n\n\n Teradata\n,Think Big Principal Data Scientist,"Chicago, IL 60661 (Near West Side area)",NaN,False
9,9,\n\n\n University of Chicago\n,Staff Scientist - Shuman Lab,"Chicago, IL",NaN,False


In [76]:
jobsdf.to_csv('jobScrape', encoding='utf-8')

In [50]:
## Data Cleaning

jobsdf.drop('index', axis=1, inplace=True)

In [72]:
for i in jobsdf['Company']:
    i = i.translate(None,"\n")
# jobsdf.replace("\n", " ", inplace = True)
#     str.strip(i)

TypeError: translate() takes exactly one argument (2 given)

In [70]:
jobsdf

,Company,Job,Location,Salary,Sponsored
0,\n Enterprise Consulting Services,Data Scientist,"Chicago, IL",NaN,True
1,\n\n HERE,Senior or Mid-level Data Scientist,"Chicago, IL",NaN,True
2,"\n\n Anthem, Inc.",Data Scientist or Data Scientist Lead(Healthca...,"Chicago, IL",NaN,True
3,"\n\n\n Anthem, Inc.\n",Data Scientist or Data Scientist Lead(Healthca...,"Chicago, IL 60601 (Loop area)",NaN,False
4,\n\n\n Northwestern University\n,Research Data Analyst,"Chicago, IL",NaN,False
5,\n\n\n Kaplan\n,Senior Data Scientist,"Chicago, IL",NaN,False
6,\n\n Azavar\n,Analyst-Data Analysis,"Chicago, IL",NaN,False
7,\n\n\n Workbridge Associates\n,Mid-Level Data Scientist,"Chicago, IL","[<nobr>$105,000 a year</nobr>]",False
8,\n\n\n Teradata\n,Think Big Principal Data Scientist,"Chicago, IL 60661 (Near West Side area)",NaN,False
9,\n\n\n University of Chicago\n,Staff Scientist - Shuman Lab,"Chicago, IL",NaN,False


Lastly, we need to clean up salary data. 
1. Some of the salaries are not yearly but hourly, these will be useful to us for now
2. The salaries are given as text and usually with ranges.

#### Filter out the salaries that are not yearly (filter those that refer to hour)

In [8]:
## YOUR CODE HERE

-remove '\n'
-remove white spaces
-convert types as needed
-parse job title data
-filter out sponsored
-fix salary

CSV save

-get median salary and create a high/low column
-fix location?
-senior/manager in job title

#### Write a function that takes a salary string and converts it to a number, averaging a salary range if necessary

In [9]:
## YOUR CODE HERE

### Save your results as a CSV

In [14]:
## YOUR CODE HERE

## Predicting salaries using Logistic Regression

#### Load in the the data of scraped salaries

In [12]:
## YOUR CODE HERE

#### We want to predict a binary variable - whether the salary was low or high. Compute the median salary and create a new binary variable that is true when the salary is high (above the median)

In [15]:
## YOUR CODE HERE

#### Thought experiment: What is the baseline accuracy for this model?

In [16]:
## YOUR CODE HERE

#### Create a Logistic Regression model to predict High/Low salary using statsmodel. Start by ONLY using the location as a feature. Display the coefficients and write a short summary of what they mean.

In [17]:
## YOUR CODE HERE

#### Create a few new variables in your dataframe to represent interesting features of a job title.
- For example, create a feature that represents whether 'Senior' is in the title 
- or whether 'Manager' is in the title. 
- Then build a new Logistic Regression model with these features. Do they add any value? 


In [18]:
## YOUR CODE HERE

#### Rebuild this model with scikit-learn.
- You can either create the dummy features manually or use the `dmatrix` function from `patsy`
- Remember to scale the feature variables as well!


In [19]:
## YOUR CODE HERE

#### Use cross-validation in scikit-learn to evaluate the model above. 
- Evaluate the accuracy, AUC, precision and recall of the model. 
- Discuss the differences and explain when you want a high-recall or a high-precision model in this scenario.

In [20]:
## YOUR CODE HERE

#### Compare L1 and L2 regularization for this logistic regression model. What effect does this have on the coefficients learned?

In [21]:
## YOUR CODE HERE

In [22]:
## YOUR CODE HERE

#### Continue to incorporate other text features from the title or summary that you believe will predict the salary and examine their coefficients

#### Take ~100 scraped entries with salaries. Convert them to use with your model and predict the salary - which entries have the highest predicted salaries?

### BONUS 

#### Bonus: Use Count Vectorizer from scikit-learn to create features from the text summaries. 
- Examine using count or binary features in the model
- Re-evaluate the logistic regression model using these. Does this improve the model performance? 
- What text features are the most valuable? 

In [23]:
## YOUR CODE HERE

In [24]:
## YOUR CODE HERE

#### Re-test L1 and L2 regularization. You can use LogisticRegressionCV to find the optimal reguarlization parameters. 
- Re-test what text features are most valuable.  
- How do L1 and L2 change the coefficients?

In [25]:
## YOUR CODE HERE